In [8]:
import pandas as pd

df = pd.read_csv('../data/wiki_text_cleaned_v1.csv')

# check if the id column is unique
print(df['id'].is_unique)

# check if the id column is a primary key
print(df['id'].is_monotonic_increasing)

True
True


In [9]:
# see if there are missing values in the page_name column
print(df['page_name'].isnull().values.any())

# print those rows where the page_name column is missing
print(df[df['page_name'].isnull()])

False
Empty DataFrame
Columns: [id, page_name, section_name, subsection_name, subsubsection_name, text, section_hierarchy, text_cleaned, word_count, is_bad]
Index: []
